# Converting the CONV-LSTM Model.h5 to CONV-LSTM Model.tflite

Converts the trained Keras model (.h5) to a TensorFlow Lite format (.tflite) for optimized inference on edge devices.

In [ ]:
model = tf.keras.models.load_model('CONV-LSTM-Model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_new_converter = True
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# TFLite Model Prediction

Uses the TensorFlow Lite interpreter to load the .tflite model, make predictions on test data, and collect predicted and true values.

In [ ]:
with open('Conv_LSTM_Model.tflite', 'wb') as f:
    f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_path="Conv_LSTM_Model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

predicted_values,true_values=[],[]

for i in range(len(X_test_lstm)):
    cnn_input_data = np.expand_dims(X_test_cnn[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], cnn_input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    predicted_values.append(output_data[0][0])
    true_values.append(y_test[i])

predicted_values = np.array(predicted_values)
true_values = np.array(true_values)

# Predictions and Evaluation

Rescales the predicted and actual values to their original range and creates a DataFrame to compare predictions with actual values.

In [ ]:
predicted_values_reshaped = predicted_values.reshape(-1, 1)
predictions_rescaled = station_scaler.inverse_transform(predicted_values_reshaped).flatten()

y_test_reshaped = true_values.reshape(-1, 1)
y_test_rescaled = station_scaler.inverse_transform(y_test_reshaped).flatten()

results = pd.DataFrame(data={'Predictions': predictions_rescaled, 'Actuals': y_test_rescaled})
print(results)

# Evaluation Metrics for TFLite

Computes RMSE, MAE, and MAPE to evaluate the performance of the optimized TFLite model on test data.

In [ ]:
opti_rmse = math.sqrt(mean_squared_error(y_test_rescaled, predictions_rescaled))
opti_mae = mean_absolute_error(y_test_rescaled, predictions_rescaled)
opti_mape = mean_absolute_percentage_error(predictions_rescaled, y_test_rescaled)

print("Optimized RMSE:", opti_rmse)
print("Optimized MAE:", opti_mae)
print("Optimized MAPE:", opti_mape)